## Nemo Guardrails - Introduction

> https://www.pinecone.io/learn/nemo-guardrails-intro/

---

In [1]:
import os
import sys
from dotenv import load_dotenv
from langchain_openai import AzureOpenAI, AzureChatOpenAI, AzureOpenAIEmbeddings

In [2]:
load_dotenv("/Users/shaunaksen/Documents/personal-projects/Natural-Language-Processing/LLM Concepts/llamaindex_tutorials/knowledge_graphs/.env")

True

In [3]:
# print (os.environ['AZURE_API_KEY'])

In [ ]:
print (sys.path)

In [5]:
from nemoguardrails import LLMRails, RailsConfig

/Users/shaunaksen/miniconda3/envs/shaunak_llm/lib/python3.10/site-packages/langchain_community/llms/__init__.py:166: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
/Users/shaunaksen/miniconda3/envs/shaunak_llm/lib/python3.10/site-packages/langchain_community/llms/__init__.py:323: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.
  warn_deprecated(


In [6]:
gpt4_turbo = AzureOpenAI(
        deployment_name="gpt-4-turbo-0125",
        model="gpt-4",
        openai_api_type="azure",
        azure_endpoint="https://ml-dev.openai.azure.com",
        openai_api_key=os.environ['AZURE_API_KEY'],
        openai_api_version="2023-03-15-preview",
        max_retries=2,
        temperature=0,
    )

chat_gpt4 =AzureOpenAI(
        deployment_name="gpt-4-32k",
        model="gpt-4-32k",
        openai_api_type="azure",
        azure_endpoint="https://ml-dev.openai.azure.com",
        openai_api_key=os.environ['AZURE_API_KEY'],
        openai_api_version="2023-03-15-preview",
        max_retries=2,
        temperature=0,
    )

In [7]:
yaml_content = f"""
instructions:
- type: general
  content: /
    Below is a conversation between a personal shopping assistant and
    a user.
models:
  - type: main
    engine: azure
    model: gpt-4
    parameters:
      openai_api_version: '2023-03-15-preview'
      azure_endpoint: "https://ml-dev.openai.azure.com"
      deployment_name: gpt-4-turbo-0125
      api_key: "{os.environ['AZURE_API_KEY']}"
      temperature: 0

"""

In [15]:
colang_content = """
# define niceties
define user express greeting
    "hello"
    "hi"
    "what's up?"

define bot express greeting
    "Hello from your friendly bot :D"

define bot ask how are you
    "How are you doing?"
    "How's it going?"
    "How are you feeling today?"

define bot offer help
    "How can i help you today?"
    "Is there anything else I can help you with?"

# this is a flow called greeting
# [user express greeting, bot express greeting, bot ask how are you] - note that all these have been defined
define flow greeting
    user express greeting
    bot express greeting
    bot ask how are you

# define limits
define user ask politics
    "what are your political beliefs?"
    "thoughts on the president?"
    "left wing"
    "right wing"

define bot answer politics
    "I'm a shopping assistant, I don't like to talk of politics."

# another flow
define flow politics
    user ask politics
    bot answer politics
    bot offer help
"""

In [16]:
# initialize rails config
config = RailsConfig.from_content(
    yaml_content=yaml_content,
    colang_content=colang_content
)

In [17]:
# create rails
rails = LLMRails(config, verbose=False)

In [18]:
res = await rails.generate_async(
    prompt="Hey there!",
)

In [19]:
print (res)

Hello from your friendly bot :D
How are you doing?


The prompt `Hey there!` activated the flow - `greeting` and so we see the responses under that

```
define flow greeting
    user express greeting
    bot express greeting
    bot ask how are you
```

In [20]:
res = await rails.generate_async(prompt="what do you think of the president?")
print(res)

I'm a shopping assistant, I don't like to talk of politics.
How can i help you today?


This prompt activated the flow - `politics` and so we see the responses under that

```
define flow politics
    user ask politics
    bot answer politics
    bot offer help
```

### Colang 101

At the core of NeMo Guardrails is the Colang modeling language. Colang is a mini-language built specifically for developing dialogue flows and safety guardrails for conversational systems.

Colang is simpler, with far fewer constructs than a typical programming language. These constructs also make Colang more flexible than standard programming languages. We describe definitions, and dialogue flows with flexible natural language (using "canonical forms" and "utterances"). Let's take a look at a straightforward Colang file:

```
define user express greeting
        "hello"
        "hi"
        "what's up?"
        
define bot express greeting
        "Hey there!"

define bot ask how are you
        "How are you doing?"

define flow greeting
        user express greeting
        bot express greeting
        bot ask how are you
```

In this Colang script, we have defined the three main types of blocks in Colang. The blocks are user message blocks (define user ...), bot message blocks (define bot ...), and flow blocks (define flow ...).

These represent the primary building blocks from which we can build dialogue flows and guardrails for our chatbots.


__Canonical Forms and Utterances__

We defined three message blocks in our Colang script. The first is a user message block defined by define user express greeting — this structured representation of a message (for both user and bot messages) is known as a canonical form.

Following the canonical form, we have multiple user utterances, which are "examples" of messages that would fit into the defined canonical form. These are "hello", "hi", and "what's up".



In [30]:
colang_content = """
# define niceties
define user express greeting
    "hello"
    "hi"
    "what's up?"

define bot express greeting
    "Hello from your friendly bot :D"

define bot ask how are you
    "How are you doing?"
    "How's it going?"
    "How are you feeling today?"

define bot offer help
    "How can i help you today?"
    "Is there anything else I can help you with?"

# this is a flow called greeting
# [user express greeting, bot express greeting, bot ask how are you] - note that all these have been defined
define flow greeting
    user express greeting
    bot express greeting
    bot ask how are you

# define limits
define user ask politics
    "why doesn't the X party care about Y?"
    "why is Meta lobbying for the X party?"
    "what are your political views?"
    "who should I vote for?"

define bot answer politics
    "I'm a research assistant, I don't like to talk of politics."

define user asks llm
    "what is the llama 2 model?"
    "tell me about Meta's new LLM"
    "what are the differences between Falcon and Llama"

define bot answer llm
    "llama 2 is an open source model by meta"
    "Falcon is much smaller and not as accurate as llama 2"


# another flow
define flow politics
    user ask politics
    bot answer politics
    bot offer help


# another flow
define flow llm
    user asks llm
    bot answer llm
    bot offer help
"""

In [31]:
# initialize rails config
config = RailsConfig.from_content(
    yaml_content=yaml_content,
    colang_content=colang_content
)

In [25]:
# create rails
rails = LLMRails(config, verbose=False)

In [28]:
res = await rails.generate_async(prompt="twhat is the best llm model?")
print(res)

perplexity is the current best llm tool
Is there anything else I can help you with?


Guardrails can flexibly decide which canonical form to use based on a user or bot-message by encoding all utterances specified within the Colang file into a semantic vector space.

![](https://cdn.sanity.io/images/vr8gru94/production/f4c238be0ec0baa83e51ff227d26e8f3df1b7c8b-2895x1710.png)

When given a user/bot-message, we encode it into the same vector space, where we can calculate semantic similarity to identify the most relevant utterances and their respective canonical form.

![](https://cdn.sanity.io/images/vr8gru94/production/f8c83210f38b76eb84a589f8231ce69d8565c112-2293x1534.png)

It's also worth noting that the canonical form itself is encoded into the utterance vector space. Therefore, we could have a functional canonical form without defining its utterances. However, utterances allow us to be more specific in what our canonical form means semantically.


Following this, we defined two bot messages with `define bot express greeting` and `define bot ask how are you`. Both of these use utterances to determine better what these messages refer to.

Finally, we define a dialogue flow with `define flow greeting`. Here we specify what steps are taken if the first user express greeting message is identified as representing some user's input message. In this flow, after the user's message, the chatbot will follow the instructions in the messages `bot express greeting`, and `bot ask how are you`.




## Variables and flows

Our Colang scripts can include variables defined using the $ character. For example, if we had a $name variable, we could refer to it in our Colang like so:


In [33]:
colang_content = """

define user greeting
    "hello"
    "hi"
    "what's up?"

define bot name greeting
    "Hey $name!"

define flow
    user greeting
    if $name
        bot name greeting
    else
        bot greeting
"""

In [34]:
# initialize rails config
config = RailsConfig.from_content(
    yaml_content=yaml_content,
    colang_content=colang_content
)

# create rails
rails = LLMRails(config)

In [35]:
messages = [
    {"role": "context", "content": ""},
    {"role": "user", "content": "Hello"},
]

In [36]:
await rails.generate_async(messages=messages)

{'role': 'assistant',
 'content': 'Hello again! How can I further assist you today?'}

In [37]:
messages = [
    {"role": "context", "content": {"name": "mini"}},
    {"role": "user", "content": "Hello"},
]

In [38]:
await rails.generate_async(messages=messages)

{'role': 'assistant', 'content': 'Hey mini!'}

Now we want to set the context from within the converstation

In [39]:
colang_content = """

define user give name
    "My name is James"
    "I'm Julio"
    "Sono Andrea"

define user greeting
    "Hey there!"
    "How are you?"
    "What's up?"

define bot name greeting
    "Hey $name!"

define flow give name
    user give name
    $name = ...
    bot name greeting

define flow
    user greeting
    if not $name
        bot ask name
    else
        bot name greeting
"""

From here, we reinitialize our rails with the new colang_content and remove the `name` parameter from our context message — let's see if Guardrails can capture the `$name` parameter from our conversation:

In [41]:
# initialize rails config
config = RailsConfig.from_content(
    yaml_content=yaml_content,
    colang_content=colang_content
)

# create rails
rails = LLMRails(config)

In [42]:
messages = [
    {"role": "context", "content": ""},
    {"role": "user", "content": "Hey there!"}
]

In [43]:
res = await rails.generate_async(messages=messages)
res

{'role': 'assistant', 'content': "What's your name?"}

Here the `define flow` flow is being triggered

In [44]:
messages += [
    res,
    {"role": "user", "content": "I'm Shaunak"}
]
res = await rails.generate_async(messages=messages)
res

{'role': 'assistant', 'content': 'Hey Shaunak!'}

Guardrails successfully identifies the `user give name` message, triggering the flow `give name`, and captures the name variable from the `$name = ...` line. Allowing the chatbot to respond with the correct name in the `bot name greeting` message — which is hardcoded as `"Hey $name!"`.

These cover the essentials of variables and flows in Colang and Guardrails. We can build more flexible yet controlled conversation AI systems with these. However, there's much more we can do. Let's move on to actions.



### Actions and Agents


In Guardrails we can add actions to our colang files. These actions allow us to execute code, for example:

```
define flow
    user ask question
    $answer = execute qa_func(prompt=$last_user_message)
    bot $answer
```


In this Colang flow we expect the user to ask some question (`user ask question`), if so we run an action (equivalent to a Python function) called `qa_func`, we also pass the `$last_user_message` (a default variable set by Guardrails) to this function via the function's prompt parameter.

The function/action returns a value which we store in the `$answer` context variable, we then tell the bot to return this answer to the user (bot $answer).

Let's take a look at an example colang file for this:

In [60]:
colang_content = """

define user ask weather
    "how is the weather today?"
    "should I wear a coat?"

define bot answer weather
    bot report weather

define flow weather
    user ask weather
    $coords = execute location_api()
    $weather = execute weather_api(coords=$coords)
    bot answer weather

# here we use the chatbot for anything else
define flow
    user ...
    bot greeting
"""

In [61]:
import requests

async def weather_api(coords: list):
    latitude, longitude = coords
    res = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params={
            "latitude": latitude,
            "longitude": longitude,
            "current_weather": "true"
        }, verify=False
    )
    weather = res.json()["current_weather"]
    weather_report = f"""The current weather is:
    temperature: {weather["temperature"]}
    windspeed: {weather["windspeed"]}
    wind direction: {weather["winddirection"]} degrees
    And it is {"daytime" if weather["is_day"] else "nightime"}"""
    return weather_report

async def location_api():
    res = requests.get("http://ip-api.com/json/")
    return res.json()['lat'], res.json()['lon']

In [62]:
res = await location_api()

In [63]:
res

(12.9634, 77.5855)

In [64]:
await weather_api((12.9634, 77.5855))

/Users/shaunaksen/miniconda3/envs/shaunak_llm/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.open-meteo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


'The current weather is:\n    temperature: 27.1\n    windspeed: 11.8\n    wind direction: 128 degrees\n    And it is nightime'

We've defined our location and weather APIs required for our bot to give us up to date advice on the weather in our current location.

In [65]:
# initialize rails config
config = RailsConfig.from_content(
    colang_content=colang_content,
    yaml_content=yaml_content
)
# create rails
rails = LLMRails(config, verbose=True)

Entered verbose mode.


We need to register the two functions as actions like so:


In [66]:
rails.register_action(
    action=location_api, name="location_api",
)
rails.register_action(
    action=weather_api, name="weather_api"
)

In [67]:
await rails.generate_async(prompt="hello")

Event UtteranceUserActionFinished {'final_transcript': 'hello'}
Event StartInternalSystemAction {'uid': '8721a25f-6031-45e1-a70f-945329eff754', 'event_created_at': '2024-03-10T15:46:01.486597+00:00', 'source_uid': 'NeMoGuardrails', 'action_name': 'create_event', 'action_params': {'event': {'_type': 'UserMessage', 'text': '$user_message'}}, 'action_result_key': None, 'action_uid': '09623d3f-e13b-45d6-9cbb-536e3b11807a', 'is_system_action': True}
Executing action create_event
Event UserMessage {'uid': '9edf3189-899b-49aa-b5ec-77bc13526449', 'event_created_at': '2024-03-10T15:46:01.486818+00:00', 'source_uid': 'NeMoGuardrails', 'text': 'hello'}
Event StartInternalSystemAction {'uid': 'a8c80df2-835f-469f-be2a-2c7893a5e032', 'event_created_at': '2024-03-10T15:46:01.487092+00:00', 'source_uid': 'NeMoGuardrails', 'action_name': 'generate_user_intent', 'action_params': {}, 'action_result_key': None, 'action_uid': '009b4528-2ca0-4c5d-8488-174e0dd50e46', 'is_system_action': True}
Executing actio

'Hello again! How can I further assist you today?'

In [68]:
await rails.generate_async(prompt="how is the weather?")

Event UtteranceUserActionFinished {'final_transcript': 'how is the weather?'}
Event StartInternalSystemAction {'uid': '3da506ea-f3a4-48c5-ba6a-16a3e5d0bff9', 'event_created_at': '2024-03-10T15:46:04.287666+00:00', 'source_uid': 'NeMoGuardrails', 'action_name': 'create_event', 'action_params': {'event': {'_type': 'UserMessage', 'text': '$user_message'}}, 'action_result_key': None, 'action_uid': '385f1e30-1d66-47f1-b001-68175ba9de83', 'is_system_action': True}
Executing action create_event
Event UserMessage {'uid': '48dd1294-cee1-4c03-83d6-bd612f5ec24c', 'event_created_at': '2024-03-10T15:46:04.287841+00:00', 'source_uid': 'NeMoGuardrails', 'text': 'how is the weather?'}
Event StartInternalSystemAction {'uid': '2ffdce26-f8f6-43e3-99e3-f9158694a3b2', 'event_created_at': '2024-03-10T15:46:04.288090+00:00', 'source_uid': 'NeMoGuardrails', 'action_name': 'generate_user_intent', 'action_params': {}, 'action_result_key': None, 'action_uid': '45d0b2fd-dbd1-4458-82b4-5b646a9cf292', 'is_system_ac

/Users/shaunaksen/miniconda3/envs/shaunak_llm/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.open-meteo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Event InternalSystemActionFinished {'uid': 'd96f8114-7fe5-4686-a599-29be6241f545', 'event_created_at': '2024-03-10T15:46:05.682257+00:00', 'source_uid': 'NeMoGuardrails', 'action_uid': '1e57089c-e2cd-4854-8987-517127978b63', 'action_name': 'weather_api', 'action_params': {'coords': '$coords'}, 'action_result_key': 'weather', 'status': 'success', 'is_success': True, 'return_value': 'The current weather is:\n    temperature: 27.1\n    windspeed: 11.8\n    wind direction: 128 degrees\n    And it is nightime', 'events': [], 'is_system_action': False, 'action_finished_at': '2024-03-10T15:46:05.682268+00:00'}
Event BotIntent {'uid': 'e4003828-4bbb-4157-b3e0-f825cff7531e', 'event_created_at': '2024-03-10T15:46:05.683147+00:00', 'source_uid': 'NeMoGuardrails', 'intent': 'answer weather'}
Event StartInternalSystemAction {'uid': '78eb2584-0f97-4ac1-8249-8910d43715b7', 'event_created_at': '2024-03-10T15:46:05.684146+00:00', 'source_uid': 'NeMoGuardrails', 'action_name': 'retrieve_relevant_chunks'

'The current weather in your area is 27.1°C with a wind speed of 11.8 km/h coming from the southeast (128 degrees). It is currently nighttime. How can I assist you further?'

In [69]:
await rails.generate_async(prompt="do I need a umbrella today?")


Event UtteranceUserActionFinished {'final_transcript': 'do I need a umbrella today?'}
Event StartInternalSystemAction {'uid': 'b7cf698a-7014-4127-979e-17d13928b622', 'event_created_at': '2024-03-10T15:49:17.237560+00:00', 'source_uid': 'NeMoGuardrails', 'action_name': 'create_event', 'action_params': {'event': {'_type': 'UserMessage', 'text': '$user_message'}}, 'action_result_key': None, 'action_uid': 'a36c5495-ede3-47a2-9b79-53195830e2a7', 'is_system_action': True}
Executing action create_event
Event UserMessage {'uid': '79bb0a9c-5401-49ab-92eb-7e87e96e5173', 'event_created_at': '2024-03-10T15:49:17.237972+00:00', 'source_uid': 'NeMoGuardrails', 'text': 'do I need a umbrella today?'}
Event StartInternalSystemAction {'uid': 'e9135788-9f2e-41a2-9fc3-27d6100e9cf2', 'event_created_at': '2024-03-10T15:49:17.238347+00:00', 'source_uid': 'NeMoGuardrails', 'action_name': 'generate_user_intent', 'action_params': {}, 'action_result_key': None, 'action_uid': '44acece9-c836-447d-bbd7-c7d36ddf5e10

/Users/shaunaksen/miniconda3/envs/shaunak_llm/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.open-meteo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Event InternalSystemActionFinished {'uid': '4096fbec-89e2-4245-af27-0d076db0e9c4', 'event_created_at': '2024-03-10T15:49:21.150593+00:00', 'source_uid': 'NeMoGuardrails', 'action_uid': '721cd03c-719e-4eee-9784-239db75e9e52', 'action_name': 'weather_api', 'action_params': {'coords': '$coords'}, 'action_result_key': 'weather', 'status': 'success', 'is_success': True, 'return_value': 'The current weather is:\n    temperature: 27.1\n    windspeed: 11.8\n    wind direction: 128 degrees\n    And it is nightime', 'events': [], 'is_system_action': False, 'action_finished_at': '2024-03-10T15:49:21.150598+00:00'}
Event BotIntent {'uid': 'ba57c052-940d-49eb-b800-fdb423341e90', 'event_created_at': '2024-03-10T15:49:21.151015+00:00', 'source_uid': 'NeMoGuardrails', 'intent': 'answer weather'}
Event StartInternalSystemAction {'uid': 'ab2fecf3-b7e9-40f5-813a-dfa72a1bea10', 'event_created_at': '2024-03-10T15:49:21.151333+00:00', 'source_uid': 'NeMoGuardrails', 'action_name': 'retrieve_relevant_chunks'

"Based on the current weather conditions, it doesn't seem like you'll need an umbrella today. The temperature is 27.1°C with a wind speed of 11.8 km/h. It's also nighttime. However, always consider checking the latest forecast or if you have plans for later that might require an umbrella."